In [1]:
import numpy as np
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as mpatches
from scipy.stats import norm
import time
from IPython.display import Image as ImageDisp
from sympy import Symbol, symbols, Matrix, sin, cos, latex#, Plot
from sympy.interactive import printing
printing.init_printing()
%pylab inline --no-import-all


Populating the interactive namespace from numpy and matplotlib


In [2]:
datafile = '2014-04-23-GNSSGroundTruth.csv'

LatDD, LonDD = np.loadtxt(datafile, delimiter=',', unpack=True, skiprows=1)

print('Read \'%s\' successfully.' % datafile)


Read '2014-04-23-GNSSGroundTruth.csv' successfully.


In [3]:
dt    = 1.0/50.0 # Sample Rate of the Measurements is 50Hz
dtGPS = 1.0/10.0 # Sample Rate of GPS is 10Hz
numstates = 7

In [4]:
datafile = '2014-04-23-GPS-IMU-Data.csv'

date, \
timem, \
millis, \
ax, \
ay, \
az, \
rollrate, \
pitchrate, \
yawrate, \
roll, \
pitch, \
yaw, \
speed, \
course, \
latitude, \
longitude, \
altitude, \
pdop, \
hdop, \
vdop, \
epe, \
fix, \
satellites_view, \
satellites_used, \
temp = np.loadtxt(datafile, delimiter=',', unpack=True, skiprows=1)

print('Read \'%s\' successfully.' % datafile)

Read '2014-04-23-GPS-IMU-Data.csv' successfully.


In [5]:
R = 6378388.0 + altitude # m
arc= 2.0*np.pi*R/360.0 # m/°

dx = arc * np.cos(latitude*np.pi/180.0) * np.hstack((0.0, np.diff(longitude))) # in m
dy = arc * np.hstack((0.0, np.diff(latitude))) # in m
dz = np.hstack((0.0, np.diff(altitude)))

mx = np.cumsum(dx)
my = np.cumsum(dy)
mz = np.cumsum(dz)

ds = np.sqrt(dx**2+dy**2+dz**2)

GPS=np.hstack((True, (np.diff(ds)>0.0).astype('bool'))) # GPS Trigger for Kalman Filter

In [6]:
acc_freq = 200

# Accelerometer Noise: acc_s = acc_g + b + w 
acc_wV = 0.0004 #(m/sec^2)^2, accelerometer white noise variance
acc_bV = 0.01 #(m/sec^2)^2, accelerometer bias 

# GPS Noise
gps_pV = 10**2 # m^2
gps_vu = 100 #m/s
gps_vV = 1**2 # (m/s)^2
gps_wpV = 1 #m^2, white noise variance of gps position
gps_wvV = 0.04**2 #(m/s)^2, white noise variance of gps velocity

In [7]:
A = np.array([[1, dt, -dt**2/2],[0,1,-dt],[0,0,1]])
T = -np.array([[dt**2],[dt],[0]])

P = np.array([[gps_pV, 0, 0],[0, gps_vV, 0],[0,0,acc_bV]])
Q = T*acc_wV*np.transpose(T)

def predict(state, control, P):
    # predicting the new state
    state_pred = A*state + T*control
    # predicting the new error 
    P_pred = A*P*np.transpose(A) + Q

    return state_pred, P_pred

In [8]:
R = np.array([[gps_wpV, 0, 0],[0, gps_wvV, 0],[0,0,0]]) # measurement covariance matrix
H = np.matrix(np.eye(3))

def correct(state_pred, measurement, P_pred):
    S = H*P_pred*np.transpose(H) + R
    K = P_pred*np.transpose(H)*inv(S) # Kalman Gain
    state_corr = state_pred + K*(measurement-H*state_pred)
    P_corr = P_pred - K*H*P_pred
    
    return state_corr, P_corr

In [13]:
Accel_noise = ax
GPS_noise = mx

# white gaussian noise with zero mean and variance = 0.0004
wV_Accel = .0004 # Accelerometer white noise variance 
wm_Accel = 0 # Accelerometer white noise mean
wN_Accel = np.array([wm_Accel, wV_Accel])

# bias with a priori statistics b_a ~ N(0,0.01)
bu_Accel = 0 # Mean of bias term
bV_Accel = .01 # Variance of bias (m/s^2)^2
bN_Accel = np.array([bu_Accel, bV_Accel])

print("white gaussian noise of accelerometer : ", wN_Accel)
print("bias of accelerometer: ",bN_Accel)


def get_white_noise_accelerometer():
    return np.asscalar(np.random.normal(wN_Accel[0],np.sqrt(wN_Accel[1]),1))

def set_bias_accelerometer():
    return np.asscalar(np.random.normal(bN_Accel[0], np.sqrt(bN_Accel[1]),1))

# a = a + w + b
def get_accelerometer_reading(i, white_noise):
    return true_accel[i] + white_noise + bu_Accel

# Takes in last state @ time = k, calculates the new acceleration state @ time = k+1
def get_pos_vel_from_accel(ilast,xlast,white_noise):
    # v_new = a_last*dt + v_last
    vAc = get_accelerometer_reading(ilast,white_noise)*dtAccel + np.asscalar(xlast[1])
    # x_new = p_last + v_last*dt + a_last*dt^2/2
    xAc = (get_accelerometer_reading(ilast, white_noise)*dtAccel**2)/2 + np.asscalar(xlast[1])*dtAccel + np.asscalar(xlast[0])
    bias = np.asscalar(xlast[2])
    return np.matrix([xAc,vAc,bias]).transpose() 

white gaussian noise of accelerometer :  [0.     0.0004]
bias of accelerometer:  [0.   0.01]


In [22]:

from numpy.linalg import inv

#Initialization
p_0 = 0 
v_0 = 0
b_0 = 0 
w_prev = 0


state = [[p_0],[v_0],[0]] # p, v, b 
control = w_prev


Kalman_pos = []
Kalman_pos.append(p_0)
Kalman_vel = []
Kalman_vel.append(v_0)
for i in range(len(ax)):

    # Prediction step 
    control = ax[i]
    state_pred, P_pred = predict(state, control, P)
    state = state_pred
    P = P_pred
    
    if GPS[i]: # Correct if GPS exists 
        measurment = mx[i]
        state_corr, P_corr = correct(state, measurment, P)
        state = state_corr
        P = P_corr
    
    
    Kalman_pos.append(state[0])
    Kalman_vel.append(state[1])

LinAlgError: Singular matrix